In [1]:
from Preprocessing import Preprocessing
from utils import Behaviour, Baseline
import Experiments
import matplotlib
matplotlib.use('Agg') # Prevent drawings in this notebook
import matplotlib.pyplot as plt
from multiprocessing import Pool
import math

In [ ]:
# bids_root = "../data/"
# preprocessing = Preprocessing(bids_root, "cache")
# preprocessing.prepare_channels()

In [ ]:
#combinations = [(3, Behaviour.SHOOT_BUTTON), (8, Behaviour.MISSILE_HIT_ENEMY)]
combinations = [(i, behaviour) for i in range(65) for behaviour in Behaviour]
combinations.sort(key=lambda x: x[0])

process_count = 20
inputs = []
step_size = math.ceil(len(combinations)/process_count)
for i in range(process_count):
    inputs.append(combinations[step_size*i:step_size*(i+1)])

with Pool(process_count) as p:
    result = p.map(Experiments.run, inputs)

100%|██████████| 23/23 [09:29<00:00, 24.74s/it]


{<Behaviour.STATUS: 'STATUS'>: {0: ([np.float64(22.54361614356225), np.float64(21.57824862583322), np.float64(17.098798968196917)], [np.float64(14.541784002404714), np.float64(13.490570354191107), np.float64(21.827923446667583)]), 1: ([np.float64(13.647716565326956), np.float64(12.968056201228293), np.float64(14.212758480205842)], [np.float64(20.30789707480072), np.float64(19.153585206288035), np.float64(15.782800371208257)]), 2: ([np.float64(25.173618237864854), np.float64(23.79645880707742), np.float64(20.863246541675906)], [np.float64(12.229292061418178), np.float64(11.863659524301674), np.float64(11.63447495682668)]), 3: ([np.float64(42.528772271854336), np.float64(40.291448361249444), np.float64(37.10926858870217)], [np.float64(14.658510025848827), np.float64(14.050841431321752), np.float64(13.301129967265245)])}, <Behaviour.SHOOT_BUTTON: 'SHOOT_BUTTON'>: {0: ([np.float64(24.884689310650778), np.float64(16.535087068030016), np.float64(16.95023817048358)], [np.float64(15.4281572055

In [ ]:
# join results
peak_to_trough = {}
for data in result:
    pass

In [ ]:
baseline_to_int = {Baseline.EXCLUDED: 0, Baseline.INCLUDED: 1, Baseline.NAIVE: 2}

def plot_peak_to_trough_differences(behaviour, baseline, mode):
    data = []
    for electrode_index, peak_to_trough_value in peak_to_troughs[behaviour].items():
        data.append((electrode_index, peak_to_trough_value[mode][baseline_to_int[baseline]]))
    title = f"Lower Peak-to-Trough Differences for {behaviour.value} ({'Raw' if mode == 0 else 'ICA'}, {baseline.value})"
    sorted_values = sorted(data, key=lambda x:x[1])[:20]
    labels, values = zip(*sorted_values)
    plt.figure()
    plt.barh(range(len(values)), values, color='skyblue')
    plt.xlabel("Peak-to-Trough Difference")
    plt.ylabel("Electrode")
    plt.yticks(range(len(labels)), labels)
    plt.title(title)
    plt.savefig(f"../Results/{title}.png")
    plt.close()    

In [ ]:
peak_to_troughs[Behaviour.COLLECT_AMMO][0][0][Baseline.EXCLUDED]

NameError: name 'peak_to_troughs' is not defined

In [ ]:
for behavior in peak_to_troughs:
    for baseline in Baseline:
        plot_peak_to_trough_differences(behavior, baseline, 0)
        plot_peak_to_trough_differences(behavior, baseline, 1)